In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Training, Tuning and Deploying a PyTorch Text Classification Model on [Vertex AI](https://cloud.google.com/vertex-ai)
## Fine-tuning pre-trained [BERT](https://huggingface.co/bert-base-cased) model for sentiment classification task 

# Overview

This example is inspired from Token-Classification [notebook](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) and [run_glue.py](https://github.com/huggingface/transformers/blob/v2.5.0/examples/run_glue.py). 
We will be fine-tuning **`bert-base-cased`** (pre-trained) model for sentiment classification task.
You can find the details about this model at [Hugging Face Hub](https://huggingface.co/bert-base-cased).

For more notebooks with the state of the art PyTorch/Tensorflow/JAX, you can explore [Hugging FaceNotebooks](https://huggingface.co/transformers/notebooks.html).

### Dataset

We will be using [IMDB movie review dataset](https://huggingface.co/datasets/imdb) from [Hugging Face Datasets](https://huggingface.co/datasets).

### Objective

How to **Build, Train, Tune and Deploy PyTorch models on [Vertex AI](https://cloud.google.com/vertex-ai)** and emphasize first class support for training and deploying PyTorch models on Vertex AI. 

### Table of Contents

This notebook covers following sections:

- [Creating Notebooks instance](#Creating-Notebooks-instance-on-Google-Cloud)
- [Training](#Training)
    - [Run Training Locally in the Notebook](#Training-locally-in-the-notebook)
    - [Run Training Job on Vertex AI](#Training-on-Vertex-AI)
        - [Training with pre-built container](#Run-Custom-Job-on-Vertex-AI-Training-with-a-pre-built-container)
        - [Training with custom container](#Run-Custom-Job-on-Vertex-AI-Training-with-custom-container)
- [Tuning](#Hyperparameter-Tuning) 
    - [Run Hyperparameter Tuning job on Vertex AI](#Run-Hyperparameter-Tuning-Job-on-Vertex-AI)
- [Deploying](#Deploying)
    - [Deploying model on Vertex AI Predictions with custom container](#Deploying-model-on-Vertex AI-Predictions-with-custom-container)

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)
* [Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)
* [Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)
* [Cloud Storage](https://cloud.google.com/storage)
* [Container Registry](https://cloud.google.com/container-registry)
* [Cloud Build](https://cloud.google.com/build) *[Optional]*

Learn about [Vertex AI Pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage Pricing](https://cloud.google.com/storage/pricing) and [Cloud Build Pricing](https://cloud.google.com/build/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Creating Notebooks instance on Google Cloud

This notebook assumes you are working with [PyTorch 1.9 DLVM](https://cloud.google.com/ai-platform/notebooks/docs/images) development environment with GPU runtime. You can create a Notebook instance using [Google Cloud Console](https://cloud.google.com/notebooks/docs/create-new) or [`gcloud` command](https://cloud.google.com/sdk/gcloud/reference/notebooks/instances/create).

```
gcloud notebooks instances create example-instance \
    --vm-image-project=deeplearning-platform-release \
    --vm-image-family=pytorch-1-9-cu110-notebooks \
    --machine-type=n1-standard-4 \
    --location=us-central1-a \
    --boot-disk-size=100 \
    --accelerator-core-count=1 \
    --accelerator-type=NVIDIA_TESLA_V100 \
    --install-gpu-driver \
    --network=default
```
***
**NOTE:** You must have GPU quota before you can create instances with GPUs. Check the [quotas](https://console.cloud.google.com/iam-admin/quotas) page to ensure that you have enough GPUs available in your project. If GPUs are not listed on the quotas page or you require additional GPU quota, [request a quota increase](https://cloud.google.com/compute/quotas#requesting_additional_quota). Free Trial accounts do not receive GPU quota by default.
***

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)
1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)
1. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3. Activate the virtual environment.
1. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.
1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.
1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Python dependencies required for this notebook are [Transformers](https://pypi.org/project/transformers/), [Datasets](https://pypi.org/project/datasets/) and [hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune) will be installed in the Notebooks instance itself.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune

We will be using [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python) to interact with Vertex AI services. The high-level `aiplatform` library is designed to simplify common data science workflows by using wrapper classes and opinionated defaults. 

#### Install Vertex AI SDK for Python

In [ ]:
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

### Restart the Kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable following APIs in your project required for running the tutorial
    - [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - [Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com)
    - [Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=containerregistry.googleapis.com)
    - [Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)
1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Enter your project ID in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud` or `google.auth`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

### Authenticate your Google Cloud account

---

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

---

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key** page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).
2. Click **Create service account**.
3. In the **Service account name** field, enter a name, and click **Create**.
4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.
5. Click *Create*. A JSON file that contains your key downloads to your local environment.
6. Enter the path to your service account key as the `GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package containing your training code to a Cloud Storage bucket. Vertex AI runs the code from this package. In this tutorial, Vertex AI also saves the trained model that results from your job in the same bucket. Using this model artifact, you can then create Vertex AI model and endpoint resources in order to serve online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # <---CHANGE THIS TO YOUR BUCKET
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = f"gs://{PROJECT_ID}aip-{get_timestamp()}"

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BUCKET_NAME}")
print(f"REGION = {REGION}")

---

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

---

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

In [ ]:
import base64
import json
import os
import random
import sys

import google.auth
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.protobuf.json_format import MessageToDict

In [ ]:
from IPython.display import HTML, display

In [ ]:
import datasets
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import ClassLabel, Sequence, load_dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          EvalPrediction, Trainer, TrainingArguments,
                          default_data_collator)

In [ ]:
print(f"Notebook runtime: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version : {torch.__version__}")
print(f"Transformers version : {datasets.__version__}")
print(f"Datasets version : {transformers.__version__}")

In [ ]:
APP_NAME = "finetuned-bert-classifier"

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Training

In this section, we will train a PyTorch model by fine-tuning pre-trained model from [Hugging Face Transformers](https://github.com/huggingface/transformers). We will train the model locally first and then on [Vertex AI training service](https://cloud.google.com/vertex-ai/docs/training/custom-training).

## Training locally in the notebook

### Loading the dataset

For this example we will use [IMDB movie review dataset](https://huggingface.co/datasets/imdb) from [Hugging Face Datasets](https://huggingface.co/datasets/) for sentiment classification task. We use the [Hugging Face Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the function `load_dataset`.


In [ ]:
dataset = load_dataset("imdb")
dataset

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
print(
    "Total # of rows in training dataset {} and size {:5.2f} MB".format(
        dataset["train"].shape[0], dataset["train"].size_in_bytes / (1024 * 1024)
    )
)
print(
    "Total # of rows in test dataset {} and size {:5.2f} MB".format(
        dataset["test"].shape[0], dataset["test"].size_in_bytes / (1024 * 1024)
    )
)

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

Using the `unique` method to extract label list. This will allow us to experiment with other datasets without hard-coding labels.

In [ ]:
label_list = dataset["train"].unique("label")
label_list

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

### Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a pre-trained Hugging Face Transformers [`Tokenizer` class](https://huggingface.co/transformers/main_classes/tokenizer.html) which tokenizes the inputs (including converting the tokens to their corresponding IDs in the pre-trained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which ensures:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pre-training this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
batch_size = 16
max_seq_length = 128
model_name_or_path = "bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)
# 'use_fast' ensure that we use fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can check type of models available with a fast tokenizer on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

**NOTE:** If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenizer with the argument `is_split_into_words=True`:

In [ ]:
example = dataset["train"][4]
print(example)

In [ ]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

Note that transformers are often pre-trained with sub-word tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
# Dataset loading repeated here to make this cell idempotent
# Since we are over-writing datasets variable
dataset = load_dataset("imdb")

# Mapping labels to ids
# NOTE: We can extract this automatically but the `Unique` method of the datasets
# is not reporting the label -1 which shows up in the pre-processing.
# Hence the additional -1 term in the dictionary
label_to_id = {1: 1, 0: 0, -1: 0}


def preprocess_function(examples):
    """
    Tokenize the input example texts
    NOTE: The same preprocessing step(s) will be applied
    at the time of inference as well.
    """
    args = (examples["text"],)
    result = tokenizer(
        *args, padding="max_length", max_length=max_seq_length, truncation=True
    )

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [label_to_id[example] for example in examples["label"]]

    return result


# apply preprocessing function to input examples
dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=True)

### Fine-tuning the model

Now that our data is ready, we can download the pre-trained model and fine-tune it.

---

***Fine Tuning*** *involves taking a model that has already been trained for a given task and then tweaking the model for another similar task. Specifically, the tweaking involves replicating all the layers in the pre-trained model including weights and parameters, except the output layer. Then adding a new output classifier layer that predicts labels for the current task. The final step is to train the output layer from scratch, while the parameters of all layers from the pre-trained model are frozen. This allows learning from the pre-trained representations and "fine-tuning" the higher-order feature representations more relevant for the concrete task, such as analyzing sentiments in this case.* 

*For the scenario in the notebook analyzing sentiments, the pre-trained BERT model already encodes a lot of information about the language as the model was trained on a large corpus of English data in a self-supervised fashion. Now we only need to slightly tune them using their outputs as features for the sentiment classification task. This means quicker development iteration on a much smaller dataset, instead of training a specific Natural Language Processing (NLP) model with a larger training dataset.*


![BERT fine-tuning](./images/bert-model-fine-tuning.png)

---


Since all our tasks are about token classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=len(label_list)
)

**NOTE:** The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pre-train the model on a masked language modeling objective and replacing it with a new head for which we don't have pre-trained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
args = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    output_dir="/tmp/cls",
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a predictions and label_ids field) and has to return a dictionary string to float.

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now we create the `Trainer` object and we are almost ready to train.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

You can add callbacks to the `trainer` object to customize the behavior of the training loop such as early stopping, reporting metrics at the end of evaluation phase or taking any decisions. In the hyperparameter tuning section of this notebook, we add a callback to `trainer` for automating hyperparameter tuning process.

We can now fine-tune our model by just calling the `train` method:

In [ ]:
trainer.train()

In [ ]:
saved_model_local_path = "./models"
!mkdir ./models

In [ ]:
trainer.save_model(saved_model_local_path)

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
history = trainer.evaluate()

In [ ]:
history

To get the other metrics computed  such as precision, recall or F1 score for each category, we can apply the same function as before on the result of the `predict` method.

### Run predictions locally with sample examples

Using the trained model, we can predict the sentiment label for an input text after applying the preprocessing function that was used during the training. We will run the predictions locally in the notebook and later show how you can deploy the model to an endpoint using [TorchServe](https://pytorch.org/serve/) on Vertex AI Predictions.

In [ ]:
model_name_or_path = "bert-base-cased"
label_text = {0: "Negative", 1: "Positive"}
saved_model_path = saved_model_local_path


def predict(input_text, saved_model_path):
    # initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    # preprocess and encode input text
    tokenizer_args = (input_text,)
    predict_input = tokenizer(
        *tokenizer_args,
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="pt",
    )

    # load trained model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path)

    # get predictions
    output = loaded_model(predict_input["input_ids"])

    # return labels
    label_id = torch.argmax(*output.to_tuple(), dim=1)

    print(f"Review text: {input_text}")
    print(f"Sentiment : {label_text[label_id.item()]}\n")

In [ ]:
# example #1
review_text = (
    """Jaw dropping visual affects and action! One of the best I have seen to date."""
)
predict_input = predict(review_text, saved_model_path)

In [ ]:
# example #2
review_text = """Take away the CGI and the A-list cast and you end up with film with less punch."""
predict_input = predict(review_text, saved_model_path)

## Training on Vertex AI

You can do local experimentation on your Notebooks instance. However, for larger datasets or models often a vertically scaled compute or horizontally distributed training is required. The most effective way to perform this task is to leverage [Vertex AI custom training service](https://cloud.google.com/vertex-ai/docs/training/custom-training) for following reasons:

- **Automatically provision and de-provision resources**: Training job on Vertex AI will automatically provision computing resources, performs the training task and ensures deletion of compute resources once the training job is finished.
- **Reusability and portability**: You can package training code with its parameters and dependencies into a container and create a portable component. This container can then be run with different scenarios such as hyperparameter tuning, different data sources and more.
- **Training at scale**: You can run a [distributed training job](https://cloud.google.com/vertex-ai/docs/training/distributed-training) with AI allowing you to train models in a cluster across multiple nodes in parallel and resulting in faster training time. 
- **Logging and Monitoring**: The training service logs messages from the job to [Cloud Logging](https://cloud.google.com/logging/docs) and can be monitored while the job is running.

In this part of the notebook, we show how to scale the training job with Vertex AI by packaging the code and create a training pipeline to orchestrate a training job. There are three steps to run a training job using [Vertex AI custom training service](https://cloud.google.com/vertex-ai/docs/training/custom-training):

- **STEP 1**: Determine training code structure - Packaging as a Python source distribution or as a custom container image
- **STEP 2**: Chose a custom training method - custom job, hyperparameter training job or training pipeline
- **STEP 3**: Run the training job

![custom-training-on-vertex-ai](./images/custom-training-on-vertex-ai.png)

#### Custom training methods

There are three types of Vertex AI resources you can create to train custom models on Vertex AI:

- **[Custom jobs](https://cloud.google.com/vertex-ai/docs/training/create-custom-job):** With a custom job you configure the settings to run your training code on Vertex AI such as worker pool specs - machine types, accelerators, Python training spec or custom container spec. 
- **[Hyperparameter tuning jobs](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning):** Hyperparameter tuning jobs automate tuning of hyperparameters of your model based on the criteria you configure such as goal/metric to optimize, hyperparameters values and number of trials to run.
- **[Training pipelines](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline):** Orchestrates custom training jobs or hyperparameter tuning jobs with additional steps after the training job is successfully completed.

Please refer to the [documentation](https://cloud.google.com/vertex-ai/docs/training/custom-training-methods) for further details.

In this notebook, we will cover Custom Jobs and Hyperparameter tuning jobs.

### Packaging the training application

Before running the training job on Vertex AI, the training application code and any dependencies must be packaged and uploaded to Cloud Storage bucket or Container Registry or Artifact Registry that your Google Cloud project can access. This sections shows how to package and stage your application in the cloud.

There are two ways to package your application and dependencies and train on Vertex AI:

1. [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) with the training code and dependencies to use with a [pre-built containers](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) on Vertex AI
2. Use [custom containers](https://cloud.google.com/ai-platform/training/docs/custom-containers-training) to package dependencies using Docker containers

**This notebook shows both packaging options to run a custom training job on Vertex AI.**

#### Recommended Training Application Structure

You can structure your training application in any way you like. However, the [following structure](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container#structure) is commonly used in Vertex AI samples, and having your project's organization be similar to the samples can make it easier for you to follow the samples.

We have two directories `python_package` and `custom_container` showing both the packaging approaches. `README.md` files inside each directory has details on the directory structure and instructions on how to run application locally and on the cloud.

```
.
├── custom_container
│   ├── Dockerfile
│   ├── README.md
│   ├── scripts
│   │   └── train-cloud.sh
│   └── trainer -> ../python_package/trainer/
├── python_package
│   ├── README.md
│   ├── scripts
│   │   └── train-cloud.sh
│   ├── setup.py
│   └── trainer
│       ├── __init__.py
│       ├── experiment.py
│       ├── metadata.py
│       ├── model.py
│       ├── task.py
│       └── utils.py
└── pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb    --> This notebook
```

1. Main project directory contains your `setup.py` file or `Dockerfile` with the dependencies. 
2. Use a subdirectory named `trainer` to store your main application module and `scripts` to submit training jobs locally or cloud
3. Inside `trainer` directory:
    - `task.py` - Main application module 1) initializes and parse task arguments (hyper parameters), and 2) entry point to the trainer
    - `model.py` -  Includes function to create model with a sequence classification head from a pre-trained model.
    - `experiment.py` - Runs the model training and evaluation experiment, and exports the final model.
    - `metadata.py` - Defines metadata for classification task such as predefined model dataset name, target labels
    - `utils.py` - Includes utility functions such as data input functions to read data, save model to GCS bucket

### Run Custom Job on Vertex AI Training with a pre-built container

Vertex AI provides Docker container images that can be run as [pre-built containers](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#available_container_images) for custom training. These containers include common dependencies used in training code based on the Machine Learning framework and framework version.

In this notebook, we are using Hugging Face Datasets and fine tuning a transformer model from Hugging Face Transformers Library for sentiment analysis task using PyTorch. We will use [pre-built container for PyTorch](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch) and package the training application code by adding standard Python dependencies - `transformers`, `datasets` and `tqdm` - in the `setup.py` file. 

![Training with Prebuilt Containers on Vertex AI Training](./images/training-with-prebuilt-containers-on-vertex-training.png)

Initialize the variables to define pre-built container image, location of training application and training module.

In [ ]:
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest"
)

PYTHON_PACKAGE_APPLICATION_DIR = "python_package"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_NAME}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)
python_module_name = "trainer.task"

Following is the `setup.py` file for the training application. The `find_packages()` function inside `setup.py` includes the `trainer` directory in the package as it contains `__init__.py` which tells [Python Setuptools](https://setuptools.readthedocs.io/en/latest/) to include all subdirectories of the parent directory as dependencies. 

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess


REQUIRED_PACKAGES = [
    'transformers',
    'datasets',
    'tqdm',
    'cloudml-hypertune'
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Vertex AI | Training | PyTorch | Text Classification | Python Package'
)

Run the following command to create a source distribution, dist/trainer-0.1.tar.gz:

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

Now upload the source distribution with training application to Cloud Storage bucket

In [ ]:
!gsutil cp {source_package_file_name} {python_package_gcs_uri}

Validate the source distribution exists on Cloud Storage bucket

In [ ]:
!gsutil ls -l {python_package_gcs_uri}

#### *[Optional]* **Run custom training job locally** 

Before submitting the job to cloud, you can run the training job locally by calling the `trainer.task` module directly

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python -m trainer.task

#### **Run custom training job on Vertex AI**

We use [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#client_libraries) to create and submit training job to the Vertex AI training service.

##### **Initialize the Vertex AI SDK for Python**

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

##### **Configure and submit Custom Job to Vertex AI Training service**

Configure a [Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) with the [pre-built container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) image for PyTorch and training code packaged as Python source distribution. 

**NOTE:** When using Vertex AI SDK for Python for submitting a training job, it creates a [Training Pipeline](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) which launches the Custom Job on Vertex AI Training service.

In [ ]:
print(f"APP_NAME={APP_NAME}")
print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-pkg-ar-{get_timestamp()}"
print(f"JOB_NAME={JOB_NAME}")

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

In [ ]:
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

model = job.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

##### **Monitoring progress of the Custom Job**

You can monitor the custom job launched from Cloud Console following the link [here](https://console.cloud.google.com/vertex-ai/training/training-pipelines/) or use gcloud CLI command [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![Monitor custom job progress in Vertex AI Training](./images/vertex-training-monitor-custom-job.png)

Validate the model artifacts written to GCS by the training code after the job completes successfully

In [ ]:
job_response = MessageToDict(job._gca_resource._pb)
gcs_model_artifacts_uri = job_response["trainingTaskInputs"]["baseOutputDirectory"][
    "outputUriPrefix"
]
print(f"Model artifacts are available at {gcs_model_artifacts_uri}")

In [ ]:
!gsutil ls -lr $gcs_model_artifacts_uri/

##### ***[Optional]*** **Submit custom job using gcloud CLI using Python source distribution**
You can submit the training job to Vertex AI training service using [`gcloud beta ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create). `gcloud` command stages your training application on GCS bucket and submits the training job.


```
gcloud beta ai custom-jobs create \
    --display-name=${JOB_NAME} \
    --region ${REGION} \
    --python-package-uris=${PACKAGE_PATH} \
    --worker-pool-spec=replica-count=1,machine-type='n1-standard-8',accelerator-type='NVIDIA_TESLA_V100',accelerator-count=1,executor-image-uri=${IMAGE_URI},python-module='trainer.task',local-package-path="../python_package/" \
    --args="--model-name","finetuned-bert-classifier","--job-dir",$JOB_DIR
```

- `worker-pool-spec` parameter defines the worker pool configuration used by the custom job. Following are the fields within `worker-pool-spec`:
    - Set the `executor-image-uri` to `us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest` for training on pre-built PyTorch v1.7 image for GPU
    - Set the `local-package-path` to the path to the training code
    - Set the `python-module` to the `trainer.task` which is the main module to start your application
    - Set the `accelerator-type` and `machine-type` to set the compute type to run the application
 
Refer [documentation](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create#--args) for further details.

The script at `./python_package/scripts/train-cloud.sh` contains the `gcloud` commands to launch the custom job and monitor the logs.

In [ ]:
!cd python_package && ./scripts/train-cloud.sh

In addition to Cloud Console, you can monitor job progress by streaming logs using [gcloud CLI](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs) by passing the job id:


```
gcloud ai custom-jobs stream-logs <job_id> --region=$REGION
```

You can validate the model artifacts written to GCS by the training code by running the following command:

```
!gsutil ls -l $JOB_DIR/
```

### Run Custom Job on Vertex AI Training with custom container

To create a [training job with custom container](https://cloud.google.com/vertex-ai/docs/training/create-custom-container?hl=hr), you define a `Dockerfile` to install or add the dependencies required for the training job. Then, you build and test your Docker image locally to verify, push the image to Container Registry and submit a Custom Job to Vertex AI Training service.

![Training with custom containers on Vertex AI](./images/training-with-custom-containers-on-vertex-training.png)

#### **Build your container using Dockerfile with Training Code and Dependencies**

In the previous section, we wrapped the training application code and dependencies as Python source distribution. An alternate way to package the training application and dependencies is to [create a custom container](https://cloud.google.com/vertex-ai/docs/training/create-custom-container?hl=hr) using Dockerfile. We create a Dockerfile with a [pre-built PyTorch container image provided by Vertex AI](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#available_container_images) as the base image, install the dependencies - `transformers`, `datasets` , `tqdm` and `cloudml-hypertune` and copy the training application code.

In [ ]:
%%writefile ./custom_container/Dockerfile

# Use pytorch GPU base image
FROM us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-10:latest

# set working directory
WORKDIR /app

# Install required packages
RUN pip install google-cloud-storage transformers datasets tqdm cloudml-hypertune

# Copies the trainer code to the docker image.
COPY ./trainer/__init__.py /app/trainer/__init__.py
COPY ./trainer/experiment.py /app/trainer/experiment.py
COPY ./trainer/utils.py /app/trainer/utils.py
COPY ./trainer/metadata.py /app/trainer/metadata.py
COPY ./trainer/model.py /app/trainer/model.py
COPY ./trainer/task.py /app/trainer/task.py

# Set up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Build the image and tag the Container Registry path (gcr.io) that you will push to.

In [ ]:
CUSTOM_TRAIN_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_gpu_train_{APP_NAME}"

In [ ]:
!cd ./custom_container/ && docker build -f Dockerfile -t $CUSTOM_TRAIN_IMAGE_URI ../python_package

#### *[Optional]* **Run Training Job Locally with Custom Container**

Run the container locally in detached mode to test. When running with machine with GPUs, you can use `--gpus all` command line flag.

In [ ]:
!docker run --gpus all -it --rm $CUSTOM_TRAIN_IMAGE_URI

#### **Run custom training job on Vertex AI with Custom Container**

Before submitting the training job to Vertex AI, push the custom container image to Google Cloud Container Registry and then submit the training job to Vertex AI. 

**NOTE:** *[Container Registry](https://cloud.google.com/container-registry) is a central repository to store, manage, and secure your Docker container images.*

##### **Push the container to Container Registry**

Push your container image with training application code and dependencies to your Container Registry. 

In [ ]:
!docker push $CUSTOM_TRAIN_IMAGE_URI

Validate the custom container image in Container Registry

In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI

##### **Initialize the Vertex AI SDK for Python**

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

##### **Configure and submit Custom Job to Vertex AI Training service**

Configure a [Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) with the [custom container](https://cloud.google.com/vertex-ai/docs/training/create-custom-container) image with training code and other dependencies

**NOTE:** When using Vertex AI SDK for Python for submitting a training job, it creates a [Training Pipeline](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) which launches the Custom Job to train on Vertex AI Training.

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-cstm-cntr-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={CUSTOM_TRAIN_IMAGE_URI}")
print(f"JOB_NAME={JOB_NAME}")

In [ ]:
# configure the job with container image spec
job = aiplatform.CustomContainerTrainingJob(
    display_name=f"{JOB_NAME}", container_uri=f"{CUSTOM_TRAIN_IMAGE_URI}"
)

In [ ]:
# define training code arguments
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

In [ ]:
# submit the custom job to Vertex AI training service
model = job.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

##### **Monitoring progress of the Custom Job**

You can monitor the custom job launched from Cloud Console following the link [here](https://console.cloud.google.com/vertex-ai/training/training-pipelines/) or use gcloud CLI command [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![Monitor custom job progress in Vertex AI Training](./images/vertex-training-monitor-custom-job-container.png)

##### ***[Optional]*** **Submit Custom Job using gcloud CLI using custom container**
You can submit the training job to Vertex AI training service using [`gcloud beta ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create) with custom container spec. `gcloud` command submits the training job and launches worker pool with the custom container image specified.


```
gcloud beta ai custom-jobs create \
    --display-name=${JOB_NAME} \
    --region ${REGION} \
    --worker-pool-spec=replica-count=1,machine-type='n1-standard-8',accelerator-type='NVIDIA_TESLA_V100',accelerator-count=1,container-image-uri=${CUSTOM_TRAIN_IMAGE_URI} \
    --args="--model-name","finetuned-bert-classifier","--job-dir",$JOB_DIR
```

- `worker-pool-spec` parameter defines the worker pool configuration used by the custom job. Following are the fields within `worker-pool-spec`:
    - Set the `container-image-uri` to the custom container image pushed to Google Cloud Container Registry for training
    - Set the `accelerator-type` and `machine-type` to set the compute type to run the application
 
Refer [documentation](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/create#--args) for further details.

The script at `./custom_container/scripts/train-cloud.sh` contains the `gcloud` commands to launch the custom job and monitor the logs.

In [ ]:
!cd custom_container && ./scripts/train-cloud.sh

In addition to Cloud Console, you can monitor job progress by streaming logs using [gcloud CLI](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs) by passing the job id:


```
gcloud ai custom-jobs stream-logs <job_id> --region=$REGION
```

You can validate the model artifacts written to GCS by the training code by running the following command:

```
!gsutil ls -l $JOB_DIR/
```

## Hyperparameter Tuning

The training application code for fine-tuning a transformer model for sentiment analysis task uses hyperparameters such as learning rate and weight decay. These hyperparameters control the behavior of the training algorithm and can have a significant effect on the performance of the resulting model. This part of the notebook show how you can automate tuning these hyperparameters with Vertex AI Training service.

We submit a [Hyperparameter Tuning job](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) to Vertex AI Training service by packaging the training application code and dependencies in a Docker container and push the container to Google Container Registry, similar to running a Custom Job on Vertex AI with Custom Container.

![Hyperparameter Tuning with Custom Containers on Vertex AI Training](./images/hp-tuning-with-custom-containers-on-vertex-training.png)

### How hyperparameter tuning works in Vertex AI?

Following are the high level steps involved in running a Hyperparameter Tuning job on Vertex AI Training service:

- You define the hyperparameters to tune the model along with the metric (or goal) to optimize
- Vertex AI runs multiple trials of your training application with the hyperparameters and limits you specified - maximum number of trials to run and number of parallel trials. 
- Vertex AI keeps track of the results from each trial and makes adjustments for subsequent trials. This requires your training application to report the metrics to Vertex AI using the Python package [`cloudml-hypertune`](https://github.com/GoogleCloudPlatform/cloudml-hypertune). 
- When the job is finished, get the summary of all the trials with the most effective configuration of values based on the criteria you configured

Refer to the [Vertex AI documentation](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) to understand how to configure and select hyperparameters for tuning, configure tuning strategy and how Vertex AI optimizes the hyperparameter tuning jobs. The default tuning strategy uses results of previous trials to inform the assignment of values in subsequent trials. 

### Changes to training application code for hyperparameter tuning

There are few [requirements](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview#the_flow_of_hyperparameter_values) to follow specific to hyperparameter tuning in Vertex AI:

1. To pass the hyperparameter values to training code, you mist define a command-line argument in the main training module for each tuned hyperparameter. Use the value passed in those arguments to set the corresponding hyperparameter in the training application's code
1. You must pass metrics from the training application to Vertex AI to evaluate the effectiveness of a trial. You can use [`cloudml-hypertune` Python package](https://github.com/GoogleCloudPlatform/cloudml-hypertune) to report metrics.

Previously, in the training application code to fine-tune the transformer model for sentiment analysis task, we instantiated [`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html) with hyperparameters passed as training arguments (`training_args`). 

```
    # Estimator arguments
    args_parser.add_argument(
        '--learning-rate',
        help='Learning rate value for the optimizers.',
        default=2e-5,
        type=float)
    args_parser.add_argument(
        '--weight-decay',
        help="""
      The factor by which the learning rate should decay by the end of the
      training.

      decayed_learning_rate =
        learning_rate * decay_rate ^ (global_step / decay_steps)

      If set to 0 (default), then no decay will occur.
      If set to 0.5, then the learning rate should reach 0.5 of its original
          value at the end of the training.
      Note that decay_steps is set to train_steps.
      """,
        default=0.01,
        type=float)

    # Enable hyperparameter
    args_parser.add_argument(
        '--hp-tune',
        default="n",
        help='Enable hyperparameter tuning. Valida values are: "y" - enable, "n" - disable')

```


These hyperparameters are passed as command line arguments to the training module `trainer.task` which are then passed to the `training_args`. Refer to `./python_package/trainer` module for training application code.

```
    # set training arguments
    training_args = TrainingArguments(
        evaluation_strategy="epoch",
        learning_rate=args.learning_rate,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.num_epochs,
        weight_decay=args.weight_decay,
        output_dir=os.path.join("/tmp", args.model_name)
    )
    
    # initialize our Trainer
    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=default_data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
```

To report metrics to Vertex AI when hyperparameter tuning is enabled, we call [`cloudml-hypertune` Python package](https://github.com/GoogleCloudPlatform/cloudml-hypertune) after the evaluation phase which is added as a [callback](https://huggingface.co/transformers/main_classes/callback.html#transformers.trainer_callback.TrainerCallback) to the `trainer`. The `trainer` objects passes the metrics computed by the last evaluation phase to the callback which will be reported by `hypertune` library to Vertex AI for evaluating trials.

```
# add hyperparameter tuning callback to report metrics when enabled
if args.hp_tune == "y":
    trainer.add_callback(HPTuneCallback("accuracy", "eval_accuracy"))

class HPTuneCallback(TrainerCallback):
    """
    A custom callback class that reports a metric to hypertuner
    at the end of each epoch.
    """
    
    def __init__(self, metric_tag, metric_value):
        super(HPTuneCallback, self).__init__()
        self.metric_tag = metric_tag
        self.metric_value = metric_value
        self.hpt = hypertune.HyperTune()
        
    def on_evaluate(self, args, state, control, **kwargs):
        print(f"HP metric {self.metric_tag}={kwargs['metrics'][self.metric_value]}")
        self.hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag=self.metric_tag,
            metric_value=kwargs['metrics'][self.metric_value],
            global_step=state.epoch)
```

### Run Hyperparameter Tuning Job on Vertex AI

Before submitting the hyperparameter tuning job to Vertex AI, push the custom container image with training application to Google Cloud Container Registry and then submit the job to Vertex AI. We will be using the same image used for running Custom Job on Vertex AI Training service.

Validate the custom container image in Container Registry

In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI

##### **Initialize the Vertex AI SDK for Python**

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

##### **Configure and submit Hyperparameter Tuning Job to Vertex AI Training service**

Configure a [Hyperparameter Tuning Job](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning) with the [custom container](https://cloud.google.com/vertex-ai/docs/training/create-custom-container) image with training code and other dependencies.

When configuring and submitting a Hyperparameter Tuning job, you need to attach a Custom Job definition with worker pool specs defining machine type, accelerators and URI for container image representing the custom container.

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-hptune-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={CUSTOM_TRAIN_IMAGE_URI}")
print(f"JOB_NAME={JOB_NAME}")

Define the training arguments with `hp-tune` argument set to `y` so that training application code can report metrics to Vertex AI

In [ ]:
training_args = [
    "--num-epochs",
    "2",
    "--model-name",
    "finetuned-bert-classifier",
    "--hp-tune",
    "y",
]

Create a **`CustomJob`** with worker pool specs to define machine types, accelerators and customer container spec with the training application code

In [ ]:
# The spec of the worker pools including machine type and Docker image
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-8",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": CUSTOM_TRAIN_IMAGE_URI, "args": training_args},
    }
]

In [ ]:
custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME, worker_pool_specs=worker_pool_specs
)

Define the `parameter_spec` as a Python dictionary object with the search space i.e. parameters to search and optimize. They key is the hyperparameter name passed as command line argument to the training code and value is the parameter specification. The spec requires to specify the hyperparameter data type as an instance of a parameter value specification. 

Refer to the [documentation](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview#hyperparameters) on selecting the hyperparaneter to tune and how to define parameter specification.

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning-rate": hpt.DoubleParameterSpec(min=1e-6, max=0.001, scale="log"),
    "weight-decay": hpt.DiscreteParameterSpec(
        values=[0.0001, 0.001, 0.01, 0.1], scale=None
    ),
}

Define the `metric_spec` with name and goal of metric to optimize. The goal specifies whether you want to tune your model to maximize or minimize the value of this metric.

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

Configure and submit a Hyperparameter Tuning Job with the Custom Job, metric spec, parameter spec and trial limits.

- **`max_trial_count`**: Maximum # of Trials run by the service. We recommend to start with a smaller value to understand the impact of the hyperparameters chosen before scaling up.
- **`parallel_trial_count`**: Number of Trials to run in parallel. We recommend to start with a smaller value as Vertex AI uses results from the previous trials to inform the assignment of values in subsequent trials. Large # of parallel trials mean these trials start without having the benefit of the results of any trials still running.
- **`search_algorithm`**: Search algorithm specified for the Study. If you do not specify an algorithm, Vertex AI by default applies Bayesian optimization to arrive at the optimal solution to search over the parameter space. 

Refer to the [documentation](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning#configuration) to understand the hyperparameter training job configuration.

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name=JOB_NAME,
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=5,
    parallel_trial_count=2,
    search_algorithm=None,
)

In [ ]:
model = hp_job.run(sync=False)

##### **Monitoring progress of the Custom Job**

You can monitor the hyperparameter tuning job launched from Cloud Console following the link [here](https://console.cloud.google.com/vertex-ai/training/hyperparameter-tuning-jobs/) or use gcloud CLI command [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)

![Monitor hyperparameter tuning job progress in Vertex AI Training](./images/vertex-training-monitor-hptuning-job-container.png)

After the job is finished, you can view and format the results of the hyperparameter tuning Trials (run by Vertex AI Training service) as a Pandas dataframe

In [ ]:
def get_trials_as_df(trials):
    results = []
    for trial in trials:
        row = {}
        t = MessageToDict(trial._pb)
        # print(t)
        row["Trial ID"], row["Status"], row["Start time"], row["End time"] = (
            t["id"],
            t["state"],
            t["startTime"],
            t.get("endTime", None),
        )

        for param in t["parameters"]:
            row[param["parameterId"]] = param["value"]

        if t["state"] == "SUCCEEDED":
            row["Training step"] = t["finalMeasurement"]["stepCount"]
            for metric in t["finalMeasurement"]["metrics"]:
                row[metric["metricId"]] = metric["value"]
        results.append(row)

    _df = pd.DataFrame(results)
    return _df

In [ ]:
df_trials = get_trials_as_df(hp_job.trials)
df_trials

Now from the results of Trials, you can pick the best performing Trial to deploy to Vertex AI Predictions

In [ ]:
# get trial id of the best run from the Trials
best_trial_id = df_trials.loc[df_trials["accuracy"].idxmax()]["Trial ID"]

In [ ]:
# get base output directory where artifacts are saved
base_output_dir = MessageToDict(hp_job._gca_resource._pb)["trialJobSpec"][
    "baseOutputDirectory"
]["outputUriPrefix"]

# get the model artifacts of the best trial id
best_model_artifact_uri = f"{base_output_dir}/{best_trial_id}"

print(
    f"Model artifacts from the Hyperparameter Tuning Job with bbest trial id {best_trial_id} are located at {best_model_artifact_uri}"
)

You can validate the model artifacts written to GCS by the training code by running the following command:

In [ ]:
!gsutil ls -r $best_model_artifact_uri/

##### ***[Optional]*** **Submit hyperparameter tuning job using gcloud CLI**
You can submit the hyperparameter tuning job to Vertex AI training service using [`gcloud beta ai hp-tuning-jobs create`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/hp-tuning-jobs/create). `gcloud` command submits the hyperparameter tuning job and launches multiple trials with worker pool based on custom container image specified and number of trials and the criteria set. The command requires hyperparameter tuning job configuration provided as configuration file in YAML format with job name.

The following cell shows how to submit a hyperparameter tuning job on Vertex AI using `gcloud` CLI: 

In [ ]:
%%bash -s $BUCKET_NAME $APP_NAME

# ========================================================
# set job parameters
# ========================================================
# PROJECT_ID: Change to your project id
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
        
# set job display name
JOB_PREFIX="finetuned-bert-classifier"
JOB_NAME=${JOB_PREFIX}-pytorch-hptune-$(date +%Y%m%d%H%M%S)
echo "Launching hyperparameter tuning job with display name as "$JOB_NAME

# BUCKET_NAME is a required parameter to run the cell.
BUCKET_NAME=$1

# APP_NAME: get application name
APP_NAME=$2

# JOB_DIR: Where to store prepared package and upload output model.
JOB_DIR=${BUCKET_NAME}/${JOB_PREFIX}/model/${JOB_NAME}

# custom container image URI
CUSTOM_TRAIN_IMAGE_URI='gcr.io/'${PROJECT_ID}'/pytorch_gpu_train_'${APP_NAME}

# ========================================================
# create hyperparameter tuning configuration file
# ========================================================
cat << EOF > ./python_package/hptuning_job.yaml

studySpec:
  metrics:
  - metricId: accuracy
    goal: MAXIMIZE
  parameters:
  - parameterId: learning-rate
    scaleType: UNIT_LOG_SCALE
    doubleValueSpec:
      minValue: 0.000001
      maxValue: 0.001
  - parameterId: weight-decay
    scaleType: SCALE_TYPE_UNSPECIFIED
    discreteValueSpec:
      values: [
          0.0001, 
          0.001, 
          0.01, 
          0.1
      ]
  measurementSelectionType: BEST_MEASUREMENT
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-standard-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1  
    replicaCount: 1
    containerSpec:
      imageUri: $CUSTOM_TRAIN_IMAGE_URI
      args: ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier", "--hp-tune", "y"]
  baseOutputDirectory: 
    outputUriPrefix: $JOB_DIR/
EOF

# ========================================================
# submit hyperparameter tuning job
# ========================================================
gcloud beta ai hp-tuning-jobs create \
   --config ./python_package/hptuning_job.yaml \
   --display-name $JOB_NAME \
   --algorithm algorithm-unspecified \
   --max-trial-count 5 \
   --parallel-trial-count 2 \
   --region=us-central1

## Deploying

Deploying a PyTorch model on [Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) requires to use a custom container that serves online predictions. You will deploy a container running [PyTorch's TorchServe](https://pytorch.org/serve/) tool in order to serve predictions from a fine-tuned transformer model from Hugging Face Transformers for sentiment analysis task. You can then use Vertex AI Predictions to classify sentiment of input texts. 

### Deploying model on Vertex AI Predictions with custom container

To use a custom container to serve predictions from a PyTorch model, you must provide Vertex AI with a Docker container image that runs an HTTP server, such as TorchServe in this case. Please refer to [documentation](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements) that describes the container image requirements to be compatible with Vertex AI Predictions.

![Serving with Custom Containers on Vertex AI Predictions](./images/serve-pytorch-model-on-vertex-predictions-with-custom-containers.png)

Essentially, to deploy a PyTorch model on Vertex AI Predictions following are the steps:

1. Package the trained model artifacts including [default](https://pytorch.org/serve/#default-handlers) or [custom](https://pytorch.org/serve/custom_service.html) handlers by creating an archive file using [Torch model archiver](https://github.com/pytorch/serve/tree/master/model-archiver)
2. Build a [custom container](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements) compatible with Vertex AI Predictions to serve the model using Torchserve
3. Upload the model with custom container image to serve predictions as a Vertex AI Model resource
4. Create a Vertex AI Endpoint and [deploy the model](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api) resource

#### **Create a custom model handler to handle prediction requests**

When predicting sentiments of the input text with the fine-tuned transformer model, it requires pre-processing of the input text and post-processing by adding name (positive/negative) to the target label (1/0) along with probability (or confidence). We create a custom handler script that is packaged with the model artifacts and TorchServe executes the code when it runs. 

Custom handler script does the following:

- Pre-process input text before sending it to the model for inference
- Customize how the model is invoked for inference
- Post-process output from the model before sending back a response

Please refer to the [TorchServe documentation](https://pytorch.org/serve/custom_service.html) for defining a custom handler.

In [ ]:
%%writefile predictor/custom_handler.py

import os
import json
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text 
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Read model serialize/pt file
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")
        
        # Load model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.model.to(self.device)
        self.model.eval()
        logger.debug('Transformer model from path {0} loaded successfully'.format(model_dir))
        
        # Ensure to use the same tokenizer used during training
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning('Missing the index_to_name.json file. Inference output will default.')
            self.mapping = {"0": "Negative",  "1": "Positive"}

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        sentences = text.decode('utf-8')
        logger.info("Received text: '%s'", sentences)

        # Tokenize the texts
        tokenizer_args = ((sentences,))
        inputs = self.tokenizer(*tokenizer_args,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors = "pt")
        return inputs

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        prediction = self.model(inputs['input_ids'].to(self.device))[0].argmax().item()

        if self.mapping:
            prediction = self.mapping[str(prediction)]

        logger.info("Model predicted: '%s'", prediction)
        return [prediction]

    def postprocess(self, inference_output):
        return inference_output


##### **Generate target label to name file** *[Optional]*

In the custom handler, we refer to a mapping file between target labels and their meaningful names that will be used to format the prediction response. Here we are mapping target label "0" as "Negative" and "1"  as "Positive". 

In [ ]:
%%writefile ./predictor/index_to_name.json

{
    "0": "Negative", 
    "1": "Positive"
}

#### **Create custom container image to serve predictions**

We will use Cloud Build to create the custom container image with following build steps:

##### **Download model artifacts**

Download model artifacts that were saved as part of the training (or hyperparameter tuning) job from Cloud Storage to local directory

In [ ]:
GCS_MODEL_ARTIFACTS_URI = best_model_artifact_uri

Validate model artifact files in the Cloud Storage bucket

In [ ]:
!gsutil ls -r $GCS_MODEL_ARTIFACTS_URI/model/

Copy files from Cloud Storage to local directory

In [ ]:
!gsutil -m cp -r $GCS_MODEL_ARTIFACTS_URI/model/ ./predictor/

In [ ]:
!ls -ltrR ./predictor/model

##### **Build the container image**

Create a Dockerfile with TorchServe as base image:

 - **`RUN`**: Installs dependencies such as `transformers`
 - **`COPY`**: Add model artifacts to `/home/model-server/` directory of the container image
 - **`COPY`**: Add custom handler script to `/home/model-server/` directory of the container image
 - **`RUN`**: Create `/home/model-server/config.properties` to define the serving configuration (health and prediction listener ports)
 - **`RUN`**: Run [Torch model archiver](https://pytorch.org/serve/model-archiver.html) to create a model archive file from the files copied into the image `/home/model-server/`. The model archive is saved in the `/home/model-server/model-store/` with name same as `<model-name>.mar`
 - **`CMD`**: Launch Torchserve HTTP server referencing the configuration properties and enables serving for the model

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM pytorch/torchserve:latest-cpu

# install dependencies
RUN python3 -m pip install --upgrade pip
RUN pip3 install transformers

USER model-server

# copy model artifacts, custom handler and other dependencies
COPY ./custom_handler.py /home/model-server/
COPY ./index_to_name.json /home/model-server/
COPY ./model/$APP_NAME/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/tokenizer.json,/home/model-server/training_args.bin,/home/model-server/tokenizer_config.json,/home/model-server/special_tokens_map.json,/home/model-server/vocab.txt,/home/model-server/index_to_name.json" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]
EOF

echo "Writing ./predictor/Dockerfile"

Build the docker image tagged with Container Registry (gcr.io) path

In [ ]:
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")

In [ ]:
!docker build \
  --tag=$CUSTOM_PREDICTOR_IMAGE_URI \
  ./predictor

#### **Run the container locally** ***[Optional]***

Before push the container image to Container Registry to use it with Vertex AI Predictions, you can run it as a container in your local environment to verify that the server works as expected

1. To run the container image as a container locally, run the following command:

In [ ]:
!docker stop local_bert_classifier
!docker run -t -d --rm -p 7080:7080 --name=local_bert_classifier $CUSTOM_PREDICTOR_IMAGE_URI
!sleep 20

2. To send the container's server a health check, run the following command:

In [ ]:
!curl http://localhost:7080/ping

If successful, the server returns the following response:

```
{
  "status": "Healthy"
}
```

3. To send the container's server a prediction request, run the following commands:

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat > ./predictor/instances.json <<END
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo 'Take away the CGI and the A-list cast and you end up with film with less punch.' | base64 --wrap=0)"
       }
     }
   ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./predictor/instances.json \
  http://localhost:7080/predictions/$APP_NAME/

This request uses a test sentence. If successful, the server returns prediction in below format:

```
    {"predictions": ["Negative"]}
```

4. To stop the container, run the following command:

In [ ]:
!docker stop local_bert_classifier

#### **Deploying the serving container to Vertex AI Predictions**

We create a model resource on Vertex AI and deploy the model to a Vertex AI Endpoints. You must deploy a model to an endpoint before using the model. The deployed model runs the custom container image to serve predictions. 

##### **Push the serving container to Container Registry**

Push your container image with inference code and dependencies to your Container Registry

In [ ]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

##### **Initialize the Vertex AI SDK for Python**

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

##### **Create a Model resource with custom serving container**

In [ ]:
VERSION = 1
model_display_name = f"{APP_NAME}-v{VERSION}"
model_description = "PyTorch based text classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

For more context on upload or importing a model, refer [documentation](https://cloud.google.com/vertex-ai/docs/general/import-model)

##### **Create an Endpoint for Model with Custom Container**

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

##### **Deploy the Model to Endpoint**

Deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

**NOTE:** This step takes few minutes to deploy the resources.

In [ ]:
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

#### **Invoking the Endpoint with deployed Model using Vertex AI SDK to make predictions**

##### **Get the Endpoint id**

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

In [ ]:
endpoint.list_models()

##### **Formatting input for online prediction**

This notebook uses [Torchserve's KServe based inference API](https://pytorch.org/serve/inference_api.html#kserve-inference-api) which is also [Vertex AI Predictions compatible format](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#prediction). For online prediction requests, format the prediction input instances as JSON with base64 encoding as shown here:

```
[
    {
        "data": {
            "b64": "<base64 encoded string>"
        }
    }
]
```

Define sample texts to test predictions

In [ ]:
test_instances = [
    b"Jaw dropping visual affects and action! One of the best I have seen to date.",
    b"Take away the CGI and the A-list cast and you end up with film with less punch.",
]

##### **Sending an online prediction request**

Format input text string and call prediction endpoint with formatted input request and get the response

In [ ]:
print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(test_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=test_instance)
    print(f"Prediction response: \n\t{prediction}")
    print("=" * 100)

##### ***[Optional]*** **Make prediction requests using gcloud CLI**
You can also call the Vertex AI Endpoint to make predictions using [`gcloud beta ai endpoints predict`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/endpoints/predict). 

The following cell shows how to make a prediction request to Vertex AI Endpoints using `gcloud` CLI: 

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"

In [ ]:
%%bash -s $REGION $endpoint_display_name

REGION=$1
endpoint_display_name=$2

# get endpoint id
echo "REGION = ${REGION}"
echo "ENDPOINT DISPLAY NAME = ${endpoint_display_name}"
endpoint_id=$(gcloud beta ai endpoints list --region ${REGION} --filter "display_name=${endpoint_display_name}" --format "value(ENDPOINT_ID)")
echo "ENDPOINT_ID = ${endpoint_id}"

# call prediction endpoint
input_text="Take away the CGI and the A-list cast and you end up with film with less punch."
echo "INPUT TEXT = ${input_text}"

prediction=$(
echo """
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo ${input_text} | base64 --wrap=0)"
       }
     }
   ]
}
""" | gcloud beta ai endpoints predict ${endpoint_id} --region=$REGION --json-request -)

echo "PREDICTION RESPONSE = ${prediction}"

## Cleaning up 

### Cleaning up training and deployment resources

To clean up all Google Cloud resources used in this notebook, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Training Jobs
- Model
- Endpoint
- Cloud Storage Bucket
- Container Images


Set flags for the resource type to be deleted

In [ ]:
delete_custom_job = False
delete_hp_tuning_job = False
delete_endpoint = True
delete_model = False
delete_bucket = False
delete_image = False

Define clients for jobs, models and endpoints

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"

client_options = {"api_endpoint": API_ENDPOINT}

# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [ ]:
# functions to create client
def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()

Define functions to list the jobs, models and endpoints starting with `APP_NAME` defined earlier in the notebook

In [ ]:
def list_custom_jobs():
    client = clients["job"]
    jobs = []
    response = client.list_custom_jobs(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            jobs.append((_row["name"], _row["displayName"]))
    return jobs


def list_hp_tuning_jobs():
    client = clients["job"]
    jobs = []
    response = client.list_hyperparameter_tuning_jobs(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            jobs.append((_row["name"], _row["displayName"]))
    return jobs


def list_models():
    client = clients["model"]
    models = []
    response = client.list_models(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            models.append((_row["name"], _row["displayName"]))
    return models


def list_endpoints():
    client = clients["endpoint"]
    endpoints = []
    response = client.list_endpoints(parent=PARENT)
    for row in response:
        _row = MessageToDict(row._pb)
        if _row["displayName"].startswith(APP_NAME):
            print(_row)
            endpoints.append((_row["name"], _row["displayName"]))
    return endpoints

#### **Deleting custom training jobs**

In [ ]:
# Delete the custom training using the Vertex AI fully qualified identifier for the custom training
try:
    if delete_custom_job:
        custom_jobs = list_custom_jobs()
        for job_id, job_name in custom_jobs:
            print(f"Deleting job {job_id} [{job_name}]")
            clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

#### **Deleting hyperparameter tuning jobs**

In [ ]:
# Delete the hyperparameter tuning jobs using the Vertex AI fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hp_tuning_job:
        hp_tuning_jobs = list_hp_tuning_jobs()
        for job_id, job_name in hp_tuning_jobs:
            print(f"Deleting job {job_id} [{job_name}]")
            clients["job"].delete_hyperparameter_tuning_job(name=job_id)
except Exception as e:
    print(e)

#### **Undeploy models and Delete endpoints**

In [ ]:
# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        endpoints = list_endpoints()
        for endpoint_id, endpoint_name in endpoints:
            endpoint = aiplatform.Endpoint(endpoint_id)
            # undeploy models from the endpoint
            print(f"Undeploying all deployed models from the endpoint {endpoint_name}")
            endpoint.undeploy_all(sync=True)
            # deleting endpoint
            print(f"Deleting endpoint {endpoint_id} [{endpoint_name}]")
            clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

#### **Deleting models**

In [ ]:
# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    models = list_models()
    for model_id, model_name in models:
        print(f"Deleting model {model_id} [{model_name}]")
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

#### **Delete contents from the staging bucket**

---

***NOTE: Everything in this Cloud Storage bucket will be DELETED. Please run it with caution.***

---

In [ ]:
if delete_bucket and "BUCKET_NAME" in globals():
    print(f"Deleting all contents from the bucket {BUCKET_NAME}")

    shell_output = ! gsutil du -as $BUCKET_NAME
    print(
        f"Size of the bucket {BUCKET_NAME} before deleting = {shell_output[0].split()[0]} bytes"
    )

    # uncomment below line to delete contents of the bucket
    # ! gsutil rm -r $BUCKET_NAME

    shell_output = ! gsutil du -as $BUCKET_NAME
    if float(shell_output[0].split()[0]) > 0:
        print(
            "PLEASE UNCOMMENT LINE TO DELETE BUCKET. CONTENT FROM THE BUCKET NOT DELETED"
        )

    print(
        f"Size of the bucket {BUCKET_NAME} after deleting = {shell_output[0].split()[0]} bytes"
    )

#### **Delete images from Container Registry**

Deletes all the container images created in this tutorial with prefix defined by variable `APP_NAME` from the registry. All associated tags are also deleted.

In [ ]:
gcr_images = !gcloud container images list --repository=gcr.io/$PROJECT_ID --filter="name~"$APP_NAME

if delete_image:
    for image in gcr_images:
        if image != "NAME":  # skip header line
            print(f"Deleting image {image} including all tags")
            !gcloud container images delete $image --force-delete-tags --quiet

### Cleaning up Notebook Environment

After you are done experimenting, you can either [STOP](https://cloud.google.com/ai-platform/notebooks/docs/shut-down) or DELETE the AI Notebook instance to prevent any  charges. If you want to save your work, you can choose to stop the instance instead.

```
# Stopping Notebook instance
gcloud notebooks instances stop example-instance --location=us-central1-a


# Deleting Notebook instance
gcloud notebooks instances delete example-instance --location=us-central1-a
```